### Riksarkivet 
* [#65](https://github.com/salgo60/Svenskaforsamlingar/issues/65)
* Denna Notebook [Riksarkivet_65_link_check](https://github.com/salgo60/Svenskaforsamlingar/tree/master/Notebook/Riksarkivet_65_link_check.ipynb)

### Jmf SCB
* [#4](https://github.com/salgo60/SCB-Wikidata/issues/4)
* denna [GetAllSCBLinks.ipynb](https://github.com/salgo60/SCB-Wikidata/blob/main/notebook/GetAllSCBLinks.ipynb)

Tanken är att kolla alla Wikipedia domäner

In [1]:
import time

from datetime import datetime

now = datetime.now()
timestamp = now.timestamp()

start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

Start: 2025-11-29 22:30:29


In [2]:
SCRIPT_NAME = "Riksarkivet_65_link_check.ipynb"
SCRIPT_URL = (
    "https://github.com/salgo60/Svenskaforsamlingar/"
    "blob/master/Notebook/Riksarkivet_65_link_check.ipynb"
)

In [3]:
def read_domains(file_path):
    print(f"[DEBUG] Reading domains from: {file_path}")
    df = pd.read_csv(file_path, header=0)   # <- skip header row
    domains_list = df.iloc[:, 0].dropna().unique().tolist()
    print(f"[DEBUG] Found {len(domains_list)} domains.")
    return domains_list


In [4]:
import requests

def fetch_sitematrix_df():
    url = "https://meta.wikimedia.org/w/api.php"
    params = {
        "action": "sitematrix",
        "format": "json"
    }
    headers = {
        "User-Agent": "salgo60-language-fetcher/1.0 (salgo60@msn.com)"
    }

    print("[DEBUG] Fetching sitematrix…")
    r = requests.get(url, params=params, headers=headers)
    r.raise_for_status()

    if "application/json" not in r.headers.get("Content-Type", ""):
        raise ValueError("Server returned non-JSON response")

    data = r.json()["sitematrix"]

    rows = []

    # --- language-specific sites ---
    for key, lang_block in data.items():
        if not key.isdigit():
            continue  # skip "count", "specials"

        lang_code = lang_block.get("code")
        lang_name = lang_block.get("name")

        for site in lang_block.get("site", []):
            rows.append({
                "lang_code": lang_code,
                "lang_name": lang_name,
                "project": site.get("project"),
                "url": site.get("url"),
                "dbname": site.get("dbname"),
                "site_name": site.get("sitename"),
                "closed": site.get("closed", False)
            })

    # --- special wikis (Wikidata, Commons, Meta, etc.) ---
    for site in data.get("specials", []):
        rows.append({
            "lang_code": "special",
            "lang_name": "special",
            "project": site.get("project"),
            "url": site.get("url"),
            "dbname": site.get("dbname"),
            "site_name": site.get("sitename"),
            "closed": site.get("closed", False)
        })

    return pd.DataFrame(rows)


In [5]:
#data

In [69]:
import requests
import pandas as pd


HEADERS = {
    "User-Agent": "salgo60-language-fetcher/2.0 (https://github.com/salgo60)"
}


df_wiki = fetch_sitematrix_df()
df_wiki["closed"] = df_wiki["closed"].fillna(False).astype(bool)

df_wiki.head()



[DEBUG] Fetching sitematrix…


,lang_code,lang_name,project,url,dbname,site_name,closed
0,aa,Qafár af,None,https://aa.wikipedia.org,aawiki,Wikipedia,False
1,aa,Qafár af,None,https://aa.wiktionary.org,aawiktionary,Wiktionary,False
2,aa,Qafár af,None,https://aa.wikibooks.org,aawikibooks,Wikibooks,False
3,ab,аԥсшәа,None,https://ab.wikipedia.org,abwiki,Авикипедиа,False
4,ab,аԥсшәа,None,https://ab.wiktionary.org,abwiktionary,Wiktionary,False


In [73]:
df_wiki = df_wiki[df_wiki["site_name"] == "Wikipedia"]

In [74]:
df_wiki.info()

<class 'pandas.core.frame.DataFrame'>
Index: 207 entries, 0 to 1061
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   lang_code  207 non-null    object
 1   lang_name  206 non-null    object
 2   project    0 non-null      object
 3   url        207 non-null    object
 4   dbname     207 non-null    object
 5   site_name  207 non-null    object
 6   closed     207 non-null    bool  
dtypes: bool(1), object(6)
memory usage: 11.5+ KB


In [75]:
df_wiki["closed"].value_counts()

closed
False    207
Name: count, dtype: int64

In [76]:
len(df_wiki)

207

In [12]:
# Finns även specials
df_wiki["closed"].value_counts()

closed
False    207
Name: count, dtype: int64

In [13]:
import os

# Get the current working directory
current_directory = os.getcwd()
print("Current Working Directory:", current_directory)



Current Working Directory: /Users/salgo/Documents/GitHub/Svenskaforsamlingar/Notebook


In [14]:
# -----------------------------------------------------------
# Fetch exturlusage entries for one lang/domain
# -----------------------------------------------------------
def fetch_exturlusage(lang, domain):
    base = f"https://{lang}.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "format": "json",
        "list": "exturlusage",
        "euquery": domain,
        "eulimit": "max"
    }
    while True:
        r = session.get(base, params=params, timeout=10)
        try:
            data = r.json()
        except ValueError:
            print(f"[WARN] {lang}: JSON decode failed")
            break

        for item in data.get("query", {}).get("exturlusage", []):
            yield {
                "lang": lang,
                "page_title": item.get("title"),
                "url": item.get("url"),
                "wiki_link": f"https://{lang}.wikipedia.org/wiki/{item.get('title').replace(' ', '_')}"
            }

        if "continue" not in data:
            break
        params.update(data["continue"])
        time.sleep(0.3)

In [15]:
import os
import time
import random
import requests
import pandas as pd
from urllib.parse import urlparse
from tqdm.notebook import tqdm
file_path_domain = "sources/domains.csv"
domains = read_domains(file_path_domain)
print(domains)


[DEBUG] Reading domains from: sources/domains.csv
[DEBUG] Found 1 domains.
['riksarkivet.se']


In [16]:

# -------------------------
# Session & helpers
# -------------------------
session = requests.Session()
session.headers.update({"User-Agent": "SCB-LinkAudit/1.0 (https://www.scb.se/)"})

# we need some filtering  
df_wiki_lang = df_wiki[df_wiki["lang_code"] != "special"] 

print("Antal Språk:",len(df_wiki_lang ))
results = []
for _, row in df_wiki_lang.iterrows():
    lang = row["lang_code"]
    url  = row["url"]
    lang_name = row["lang_name"]
    before = len(results)
    #print(lang, url, lang_name,domains)
    for entry in fetch_exturlusage(lang, domains):
        results.append(entry)     
    after = len(results) 
    links = after-before
    print(lang, url, lang_name," - ", links)
    


Antal Språk: 184
aa https://aa.wikipedia.org Qafár af  -  0
ace https://ace.wikipedia.org Acèh  -  0
af https://af.wikipedia.org Afrikaans  -  6
ak https://ak.wikipedia.org None  -  0
ami https://ami.wikipedia.org Pangcah  -  0
an https://an.wikipedia.org aragonés  -  0
ast https://ast.wikipedia.org asturianu  -  105
av https://av.wikipedia.org авар  -  0
avk https://avk.wikipedia.org Kotava  -  0
ay https://ay.wikipedia.org Aymar aru  -  0
bar https://bar.wikipedia.org Boarisch  -  0
bbc https://bbc.wikipedia.org Batak Toba  -  0
bcl https://bcl.wikipedia.org Bikol Central  -  7
bi https://bi.wikipedia.org Bislama  -  0
bm https://bm.wikipedia.org bamanankan  -  0
bo https://bo.wikipedia.org བོད་ཡིག  -  0
br https://br.wikipedia.org brezhoneg  -  0
bs https://bs.wikipedia.org bosanski  -  1
btm https://btm.wikipedia.org Batak Mandailing  -  0
bug https://bug.wikipedia.org Basa Ugi  -  0
bxr https://bxr.wikipedia.org буряад  -  0
cbk-zam https://cbk-zam.wikipedia.org Chavacano de Zambo

In [17]:
domains

['riksarkivet.se']

In [18]:
df_wiki.head()

,lang_code,lang_name,project,url,dbname,site_name,closed
0,aa,Qafár af,None,https://aa.wikipedia.org,aawiki,Wikipedia,False
5,ace,Acèh,None,https://ace.wikipedia.org,acewiki,Wikipedia,False
7,af,Afrikaans,None,https://af.wikipedia.org,afwiki,Wikipedia,False
11,ak,None,None,https://ak.wikipedia.org,akwiki,Wikipedia,False
18,ami,Pangcah,None,https://ami.wikipedia.org,amiwiki,Wikipedia,False


In [19]:
df_riksarkivet = pd.DataFrame(results)

In [20]:
len(df_riksarkivet)

63040

In [21]:
df_riksarkivet 

,lang,page_title,url,wiki_link
0,af,Erik die Oorwinnaar,https://sok.riksarkivet.se/sbl/Presentation.as...,https://af.wikipedia.org/wiki/Erik_die_Oorwinnaar
1,af,Vlag van Swede,https://riksarkivet.se/sveriges-flaggas-farger,https://af.wikipedia.org/wiki/Vlag_van_Swede
2,af,Vlag van Swede,https://riksarkivet.se/vimpel,https://af.wikipedia.org/wiki/Vlag_van_Swede
3,af,Vlag van Swede,https://riksarkivet.se/flaggningstider,https://af.wikipedia.org/wiki/Vlag_van_Swede
4,af,Vlag van Swede,https://riksarkivet.se/flaggdagar,https://af.wikipedia.org/wiki/Vlag_van_Swede
...,...,...,...,...
63035,zh,厄勒布鲁城堡,https://sok.riksarkivet.se/sbl/Presentation.as...,https://zh.wikipedia.org/wiki/厄勒布鲁城堡
63036,zh,瑪麗·約翰遜,https://sok.riksarkivet.se/sbl/Presentation.as...,https://zh.wikipedia.org/wiki/瑪麗·約翰遜
63037,zh,瑪麗·約翰遜,https://sok.riksarkivet.se/Sbl/Mobil/Artikel/5856,https://zh.wikipedia.org/wiki/瑪麗·約翰遜
63038,zh,拉尔斯·约翰·耶塔,https://sok.riksarkivet.se/Sbl/Presentation.as...,https://zh.wikipedia.org/wiki/拉尔斯·约翰·耶塔


In [22]:
import pandas as pd

# --- Stats ---
total_links = len(df_riksarkivet)
total_unique_links = df_riksarkivet['url'].nunique()
num_languages = df_riksarkivet['lang'].nunique()
langs_sorted = df_riksarkivet['lang'].value_counts()

print("Total links:", total_links)
print("Total unique links:", total_unique_links)
print("Number of languages:", num_languages)
print("\nLanguages with most links:")
print(langs_sorted.to_string())


Total links: 63040
Total unique links: 43117
Number of languages: 44

Languages with most links:
lang
sv        52806
en         3815
no         2365
fi          927
de          873
nl          521
es          287
ro          235
pl          170
da          156
tt          151
it          133
id          120
ast         105
ja           77
vi           46
nn           37
zh           36
eu           34
gl           26
ha           19
se           15
sq           11
ms           10
bcl           7
tl            7
ia            7
af            6
sw            6
smn           5
simple        5
fo            5
ig            3
ilo           2
ie            2
is            2
frr           1
new           1
nds           1
so            1
kv            1
diq           1
kaa           1
bs            1


In [23]:
from urllib.parse import urlparse
import pandas as pd

# Extract domains
df_riksarkivet['domain'] = df_riksarkivet['url'].apply(lambda u: urlparse(u).netloc)

# Count links grouped by domain
links_per_domain = df_riksarkivet['domain'].value_counts()

print("Links per domain:")
print(links_per_domain.to_string())


Links per domain:
domain
sok.riksarkivet.se                     61754
riksarkivet.se                           542
www.nad.riksarkivet.se                   381
www.riksarkivet.se                       180
webbutik.riksarkivet.se                   64
arkis2dok.riksarkivet.se                  63
toramaptest.riksarkivet.se                17
forvaltningshistorik.riksarkivet.se       16
filer.riksarkivet.se                      13
data.riksarkivet.se                        5
www.sok.riksarkivet.se                     3
lbiiif.riksarkivet.se                      2


In [24]:
df_riksarkivet = df_riksarkivet.rename(columns={
    "lang": "Språk",
    "page_title": "Wikipedia-sida",
    "url": "Extern länk",
    "wiki_link": "Wikipedia-länk",
    "domain": "Domän"
})


In [25]:
import pandas as pd

# --- Stats ---
total_links = len(df_riksarkivet)
total_unique_links = df_riksarkivet['Extern länk'].nunique()
num_languages = df_riksarkivet['Språk'].nunique()
langs_sorted = df_riksarkivet['Språk'].value_counts()

print("Total links:", total_links)
print("Total unique links:", total_unique_links)
print("Number of languages:", num_languages)
print("\nLanguages with most links:")
print(langs_sorted.to_string())


Total links: 63040
Total unique links: 43117
Number of languages: 44

Languages with most links:
Språk
sv        52806
en         3815
no         2365
fi          927
de          873
nl          521
es          287
ro          235
pl          170
da          156
tt          151
it          133
id          120
ast         105
ja           77
vi           46
nn           37
zh           36
eu           34
gl           26
ha           19
se           15
sq           11
ms           10
bcl           7
tl            7
ia            7
af            6
sw            6
smn           5
simple        5
fo            5
ig            3
ilo           2
ie            2
is            2
frr           1
new           1
nds           1
so            1
kv            1
diq           1
kaa           1
bs            1


In [26]:
from datetime import date
import os

# Sätt datum
today = date.today().strftime("%Y_%m_%d")

# Se till att katalogen finns
os.makedirs("results", exist_ok=True)

# Bygg filnamn
outfile = f"results/links_riksarkivet_{today}.csv"

# Exportera
df_riksarkivet.to_csv(outfile, index=False, encoding="utf-8")

print(f"[OK] Exported {len(df_riksarkivet)} rows to {outfile}")


[OK] Exported 63040 rows to results/links_riksarkivet_2025_11_29.csv


In [27]:
print(df_riksarkivet.columns.tolist())

['Språk', 'Wikipedia-sida', 'Extern länk', 'Wikipedia-länk', 'Domän']


In [28]:
from urllib.parse import urlparse
import pandas as pd

# Extract domain
df_riksarkivet['domain'] = df_riksarkivet['Extern länk'].apply(lambda u: urlparse(u).netloc)

# Count links grouped by domain
links_per_domain = df_riksarkivet['domain'].value_counts()

print("Links per domain:")
print(links_per_domain.to_string())


Links per domain:
domain
sok.riksarkivet.se                     61754
riksarkivet.se                           542
www.nad.riksarkivet.se                   381
www.riksarkivet.se                       180
webbutik.riksarkivet.se                   64
arkis2dok.riksarkivet.se                  63
toramaptest.riksarkivet.se                17
forvaltningshistorik.riksarkivet.se       16
filer.riksarkivet.se                      13
data.riksarkivet.se                        5
www.sok.riksarkivet.se                     3
lbiiif.riksarkivet.se                      2


In [29]:

def get_git_commit():
    try:
        return subprocess.check_output(
            ["git", "rev-parse", "--short", "HEAD"],
            stderr=subprocess.DEVNULL
        ).decode().strip()
    except Exception:
        return "unknown"


In [30]:
GITHUB_REPO = "https://github.com/salgo60/Svenskaforsamlingar"
commit_hash = get_git_commit()
commit_url = f"{GITHUB_REPO}/commit/{commit_hash}"


In [31]:
script_name = SCRIPT_NAME

In [32]:
script_name

'Riksarkivet_65_link_check.ipynb'

In [41]:
print(df_riksarkivet.columns.tolist())

['Språk', 'Wikipedia-sida', 'Extern länk', 'Wikipedia-länk', 'Domän', 'domain']


In [57]:
# --- Stats ---
total_links = len(df_riksarkivet)
total_unique_links = df_riksarkivet["Extern länk"].nunique()
langs_with_hits = sorted(df_riksarkivet["Språk"].unique())

num_languages_found = len(langs_with_hits)
num_languages_checked = len(df_wiki)        # alla språk som genomsöktes
num_languages_found = df_riksarkivet['Språk'].nunique()

In [58]:
from pathlib import Path
from datetime import date
import pandas as pd

def save_sortable_html_df_riksarkivet(
    df,
    out_dir="results",
    domains=None,
    issue_url="https://github.com/salgo60/Svenskaforsamlingar/issues/65",
):
    out_dir = Path(out_dir)
    out_dir.mkdir(exist_ok=True)

    today = date.today().strftime("%Y_%m_%d")
    out_path = out_dir / f"links_riksarkivet_{today}.html"
    rerun_ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Metadata
    commit_hash = get_git_commit()

    domains = domains or [] 

    # Gör länkar klickbara om de finns
    df = df.copy()  
    if "Wikipedia-länk" in df.columns:
        df["Wikipedia-länk"] = df["Wikipedia-länk"].apply(
            lambda x: f'<a href="{x}" target="_blank">{x}</a>' if pd.notna(x) else ""
        )
    if "Extern länk" in df.columns:
        df["Extern länk"] = df["Extern länk"].apply(
            lambda x: f'<a href="{x}" target="_blank">{x}</a>' if pd.notna(x) else ""
        )

    html_table = df.to_html(
        classes="pivot",
        border=0,
        escape=False,   # viktigt för klickbara länkar
        index=False
    )
   
    css = """
    <style>
      body {
        font-family: Arial, sans-serif;
        margin: 20px;
      }
      table.pivot {
        border-collapse: collapse;
        width: 100%;
        font-size: 12px;
      }
      table.pivot th, table.pivot td {
        border: 1px solid #999;
        padding: 6px 8px;
        text-align: left;
        vertical-align: top;
        white-space: normal;
      }
      table.pivot th {
        cursor: pointer;
        background: #f2f2f2;
      }
      table.pivot th:hover {
        background: #e2e2e2;
      } 
      table.pivot thead th {
          position: sticky;
          top: 0;
          background: #f2f2f2;
          z-index: 2;
        }
        
        /* Sorting indicators */
        table.pivot th::after {
          content: "";
          float: right;
          opacity: 0.4;
        }
        
        table.pivot th.sorted-asc::after {
          content: " ▲";
        }
        
        table.pivot th.sorted-desc::after {
          content: " ▼";
        }
          .meta {
            background: #f8f8f8;
            border: 1px solid #ccc;
            padding: 12px;
            margin-bottom: 20px;
            font-size: 13px;
          }
          .meta h2 {
            margin-top: 0;
          }
          .meta ul {
            margin: 0;
            padding-left: 18px;
          }
    </style>
    """

    js = """
    <script>
    document.addEventListener('DOMContentLoaded', () => {
        document.querySelectorAll("table.pivot th").forEach((header, colIndex) => {
            header.addEventListener("click", () => {
                const table = header.closest("table");
                const tbody = table.querySelector("tbody");
                const rows = Array.from(tbody.querySelectorAll("tr"));
                const asc = !header.classList.contains("sorted-asc");

                rows.sort((a, b) => {
                    const A = a.children[colIndex].innerText.trim();
                    const B = b.children[colIndex].innerText.trim();

                    const numA = parseFloat(A.replace(",", ".")) || null;
                    const numB = parseFloat(B.replace(",", ".")) || null;

                    if (numA !== null && numB !== null) {
                        return asc ? numA - numB : numB - numA;
                    }
                    return asc ? A.localeCompare(B) : B.localeCompare(A);
                });

                table.querySelectorAll("th").forEach(th =>
                    th.classList.remove("sorted-asc","sorted-desc")
                );
                header.classList.add(asc ? "sorted-asc" : "sorted-desc");

                rows.forEach(row => tbody.appendChild(row));
            });
        });
    });
    </script>
    """
    meta_html = f"""
    <div class="meta">
      <h2>Metadata</h2>
    
      <p><strong>Rerun:</strong> {rerun_ts}</p>
      <p><strong>Script:</strong>
         <a href="{SCRIPT_URL}" target="_blank">{SCRIPT_NAME}</a>
      </p>
      <p><strong>Issue:</strong>
         <a href="{issue_url}" target="_blank">{issue_url.split('/')[-1]}</a>
      </p>
    
      <p><strong>Datakällor:</strong></p>
      <div class="meta-indent">
        Wikidata<br>
        MediaWiki API – exturlusage
      </div>
    
      <p><strong>Parametrar:</strong></p>
      <div class="meta-indent">
        Språkfilter: exkluderar "special"
      </div>
    
      <p><strong>Antal språk genomsökta:</strong> {num_languages_checked}</p>
      <p><strong>Antal språk med träffar:</strong> {num_languages_found}</p>
      <p><strong>Wikipedia-språk med träffar:</strong> {", ".join(langs_with_hits)}</p>
      <p><strong>Totala länkar:</strong> {total_links}</p>
      <p><strong>Unika länkar:</strong> {total_unique_links}</p>
    </div>
    """
    
    html = f"""
    <html>
    <head>
      <meta charset="utf-8">
      <title>Riksarkivet links in Wikipedia</title>
      {css}
    </head>
    <body>
      <h1>Wikipedia → Riksarkivet 2</h1>
      {meta_html}
      <p>Sorterbar tabell. Klicka på kolumnrubriker för sortering.</p>
      {html_table}
      {js}
    </body>
    </html>
    """

    out_path.write_text(html, encoding="utf-8")
    print(f"✅ HTML skapad: {out_path}")


In [59]:

save_sortable_html_df_riksarkivet(df_riksarkivet)


✅ HTML skapad: results/links_riksarkivet_2025_11_29.html


In [62]:
 # End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time# Bygg audit-lager för den här etappen

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
minutes, seconds = divmod(elapsed_time, 60)
print("Total time elapsed: {:02.0f} minutes {:05.2f} seconds".format(minutes, seconds))


Date: 2025-11-29 23:43:21
Total time elapsed: 72 minutes 52.07 seconds
